In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Start Spark Session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('TreeEnsembles').getOrCreate()

### Read CSV Data

In [3]:
df = spark.read.csv('College.csv', inferSchema=True, header=True)

### Data Exploration

In [4]:
df.count()

777

In [5]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
df.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [7]:
df.show(10, truncate=False)

+----------------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|School                      |Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+----------------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian University|Yes    |1660|1232  |721   |23       |52       |2885       |537        |7440    |3300      |450  |2200    |70 |78      |18.1     |12         |7041  |60       |
|Adelphi University          |Yes    |2186|1924  |512   |16       |29       |2683       |1227       |12280   |6450      |750  |1500    |29 |30      |12.2     |16         |10527 |56       |
|Adrian College              |Yes    |1428|1097  |336  

### Feature Engineering

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate','Room_Board','Books',
                                       'Personal','PhD','Terminal','S_F_Ratio','perc_alumni','Expend','Grad_Rate'], outputCol = 'features')
output = assembler.transform(df)

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
indexer = StringIndexer(inputCol='Private', outputCol='label')
output_fixed = indexer.fit(output).transform(output)

In [12]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



In [13]:
final_data = output_fixed.select('features', 'label')

### Train Test Split

In [14]:
train_data, test_data = final_data.randomSplit([0.8, 0.2])

### Tree Ensemble Methods

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [16]:
DT = DecisionTreeClassifier(labelCol='label', featuresCol='features')
RF = RandomForestClassifier(labelCol='label', featuresCol='features')
GBT = GBTClassifier(labelCol='label', featuresCol='features')

In [17]:
DT_model = DT.fit(train_data)
RF_model = RF.fit(train_data)
GBT_model = GBT.fit(train_data)

### Feature Importances

In [18]:
DT_model.featureImportances

SparseVector(17, {1: 0.0072, 5: 0.3195, 6: 0.0298, 7: 0.5465, 9: 0.0079, 11: 0.0226, 13: 0.0427, 14: 0.0238})

In [19]:
RF_model.featureImportances

SparseVector(17, {0: 0.0426, 1: 0.0351, 2: 0.0293, 3: 0.0134, 4: 0.0052, 5: 0.2163, 6: 0.0755, 7: 0.2908, 8: 0.0327, 9: 0.009, 10: 0.0221, 11: 0.0109, 12: 0.0228, 13: 0.0788, 14: 0.0432, 15: 0.0465, 16: 0.0259})

In [20]:
GBT_model.featureImportances

SparseVector(17, {0: 0.0594, 1: 0.0151, 2: 0.008, 3: 0.0308, 4: 0.0182, 5: 0.2899, 6: 0.0436, 7: 0.3305, 8: 0.0192, 9: 0.008, 10: 0.0247, 11: 0.0201, 12: 0.0179, 13: 0.0457, 14: 0.0342, 15: 0.0118, 16: 0.023})

### Grid Search

In [21]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [22]:
paramGrid_DT = ParamGridBuilder() \
               .addGrid(DT.maxDepth, [1, 3]) \
               .build()
            
paramGrid_RF = ParamGridBuilder() \
               .addGrid(RF.numTrees, [64, 128]) \
               .addGrid(RF.maxDepth, [1, 3]) \
               .build()

paramGrid_GBT = ParamGridBuilder() \
                .addGrid(GBT.stepSize, [0.1, 0.01]) \
                .build()

In [23]:
from pyspark.ml import Pipeline

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [30]:
crossval_DT = CrossValidator(estimator=Pipeline(stages=[DT]), 
                            estimatorParamMaps=paramGrid_DT,
                            evaluator=MulticlassClassificationEvaluator(),
                            numFolds=2)

In [31]:
crossval_RF = CrossValidator(estimator=Pipeline(stages=[RF]),
                          estimatorParamMaps=paramGrid_RF,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=2)

In [32]:
crossval_GBT = CrossValidator(estimator=Pipeline(stages=[GBT]),
                          estimatorParamMaps=paramGrid_GBT,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=2)